## Linear Regression

$$
f(x) = \beta_0 + \sum^p_{J=1}x_j \beta_j 
$$
We want to Minimize the residual sum of squares. To do so, first we need to compute it.
$$
\begin{aligned}
S(\beta) &= \sum^N_{i = 1}(y_i - f(x_i))^2 \\
        &= \sum^N_{i = 1} \big( y_i - \beta_0 - \sum^p_{j=1}x_{ij}\beta_j \big)^2
\end{aligned}
$$
* We have to add one more column with value 1 constant to multiply with the $\beta_0$, by doing so, the formula become:
$$
\begin{aligned}
S(\beta) &= \sum^N_{i=1} \big(y_i - \sum^p_{\color{red}{j=0}}x_{ij}\beta_j \big) ^2 \\
&= (y - X\beta)^T (y - X\beta) \\
&= y^Ty - y^T X\beta - \beta^T X^T y + \beta^T X^T X \beta
\end{aligned}
$$
* Minimizing and getting the estimated $\beta$ formula
$$
\begin{aligned}
\frac{\partial S}{\partial \beta} &= \frac{\partial (y^Ty - y^T X\beta - \beta^T X^T y + \beta^T X^T X \beta)}{\partial \beta} = 0 \\
&= -2X^T y + 2X^T X \beta = 0 \\
&= X^T X \beta = X^T y \\
\hat{\beta} &= (X^T X)^{-1} X^T y
\end{aligned}
$$
* To predic the target value to a new vector x:
$$
\hat{y} = \hat{f}(x) = (1:x)^T \hat{\beta}
$$

In [23]:
import numpy as np

In [199]:
X = np.array([[-1.75,  1.15,  0.98,  0.22, -0.19, -0.46, -0.58,  0.67, -0.53,-0.44],
                 [3.34, 2.75, 3.51, 1.93, 3.26, 3.44, 3.82, 2.9 , 4.03, 1.88]]).T
X

array([[-1.75,  3.34],
       [ 1.15,  2.75],
       [ 0.98,  3.51],
       [ 0.22,  1.93],
       [-0.19,  3.26],
       [-0.46,  3.44],
       [-0.58,  3.82],
       [ 0.67,  2.9 ],
       [-0.53,  4.03],
       [-0.44,  1.88]])

In [200]:
y = np.array([-19.56,  10.54,   5.53,   0.5 ,  -5.24,  -7.54,  -9.71,   5.26, -10.69,  -5.1])
y

array([-19.56,  10.54,   5.53,   0.5 ,  -5.24,  -7.54,  -9.71,   5.26,
       -10.69,  -5.1 ])

In [270]:
class LinearRegression():
    def __init__(self):
        self.beta = None
    
    def fit(self, X, y, pinv=False):
        X = np.array(X)
        y = np.array(y)
        X = np.insert(X, 0, np.ones(X.shape[0]), axis = 1)
        XTX = X.T @ X #can use this operator
        if not pinv:
            inverse = np.linalg.inv(XTX)
        else:
            inverse = np.linalg.pinv(XTX)
        self.beta = np.dot(np.dot(inverse, X.T), y) # or np.dot, which may be more confusing with more operations
        return self.beta
    
    def predict(self, X):
        return self.beta[0] + np.dot(X,self.beta[1:])

In [202]:
lr = LinearRegression()

In [203]:
lr.fit(X, y) #same results of beta as the professor

array([ 2.90978614,  9.91248941, -1.81105788])

In [204]:
lr.predict([-1.75, 3.34])

-20.48600365967664

In [205]:
def mse(y, y_pred):
    return np.mean((y - y_pred)**2) 

In [206]:
y[0]

-19.56

In [207]:
mse(y[0], lr.predict([-1.75, 3.34]))

0.857482777734534

### Correlated columns

In [208]:
X2 = np.ones((10,2))

In [209]:
X2[:,1] = X2[:,1] * 2

In [210]:
X2

array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]])

In [211]:
y = X2.sum(axis=1) + np.random.random((y.shape[0])) * 0.01
y

array([3.00545806, 3.00317059, 3.00302619, 3.00737119, 3.00526157,
       3.0016538 , 3.00949069, 3.00620551, 3.00196514, 3.00935995])

In [212]:
lr.fit(X2, y)

LinAlgError: Singular matrix

* Erros due to correlated columns, one way to go through is calculating the pseudo inverse

In [213]:
lr.fit(X2, y, pinv=True)

array([0.50088271, 0.50088271, 1.00176542])

In [214]:
lr.predict([1, 2])

3.0052962679438444

### real database

In [216]:
from sklearn.datasets import load_iris, load_breast_cancer

In [252]:
np.random.seed(42)

In [253]:
X,y = load_breast_cancer(return_X_y=True)

In [254]:
X.shape

(569, 30)

In [255]:
y.shape

(569,)

In [256]:
breast_cancer_dataset = np.insert(X, X.shape[1], y, axis=1)
breast_cancer_dataset.shape

(569, 31)

In [259]:
np.random.shuffle(breast_cancer_dataset)

In [261]:
breast_cancer_dataset.shape

(569, 31)

In [262]:
train_set = breast_cancer_dataset[:530]
train_set.shape

(530, 31)

In [264]:
test_set = breast_cancer_dataset[530:]
test_set.shape

(39, 31)

In [266]:
X_train, y_train = train_set[:,:30], train_set[:,-1]
print(X_train.shape, "\n",y_train.shape)

(530, 30) 
 (530,)


In [273]:
X_test, y_test = test_set[:,:30], test_set[:,-1]
print(X_test.shape, "\n",y_test.shape)

(39, 30) 
 (39,)


In [271]:
lr = LinearRegression()

In [272]:
lr.fit(X_train, y_train)

array([ 3.02323062e+00,  2.46273863e-01, -2.76441109e-03, -1.89904186e-02,
       -6.14673229e-04, -4.14077572e-01,  4.30596685e+00, -1.57533799e+00,
       -2.12308381e+00, -3.33930009e-01, -8.85551822e-01, -1.02338247e-01,
        9.70060079e-03,  1.85577029e-02, -2.36682280e-04, -1.76533389e+01,
       -1.06451020e+00,  3.44397392e+00, -9.55186262e+00, -4.63486624e+00,
        1.08095837e+01, -2.48096068e-01, -7.04847878e-03,  1.88706404e-03,
        1.24683590e-03, -6.67465509e-02,  7.41878972e-02, -4.41674584e-01,
       -4.65434343e-01, -2.48576468e-01, -4.46429797e+00])

In [275]:
y_pred = lr.predict(X_test)

In [276]:
mse(y_test, y_pred)

0.08315075211389102

In [277]:
for i in range(len(y_test)):
    print(f"real = {y_test[i]} predicted = {y_pred[i]}")

real = 0.0 predicted = -0.08556535396556209
real = 0.0 predicted = -0.0005558405745942352
real = 0.0 predicted = 0.15803823404638262
real = 0.0 predicted = 0.22466514182367492
real = 0.0 predicted = -0.14242774652906398
real = 1.0 predicted = 0.9021469086257117
real = 0.0 predicted = 0.31638057304628386
real = 1.0 predicted = 0.9720883109773069
real = 1.0 predicted = 0.9409904128031052
real = 0.0 predicted = 0.340235160007198
real = 0.0 predicted = -0.3588154817013307
real = 1.0 predicted = 0.7404432020247502
real = 0.0 predicted = 0.5889532198356271
real = 0.0 predicted = 0.322389083330739
real = 1.0 predicted = 0.9553608818991282
real = 0.0 predicted = 0.004331576830676642
real = 0.0 predicted = -0.26581300008606057
real = 0.0 predicted = 0.4281159092585738
real = 1.0 predicted = 1.0650340906225462
real = 1.0 predicted = 1.0180783286607578
real = 1.0 predicted = 0.9175272520497257
real = 0.0 predicted = 0.5823055359144012
real = 0.0 predicted = -0.5518712574971305
real = 0.0 predicte

### coeficient significance

In [ ]:
def variance(p, y, y_pred):
    